# Problem Statement 

There are six end customers, each with a known demand for a product.  Customer demand can be satisfied from a set of four depots, or directly from a set of two factories.  Each depot can support a **maximum volume of product** moving through it, and each factory can produce a **maximum amount of product**.  There are known costs associated with transporting the product, from a factory to a depot, from a depot to a customer, or from a factory directly to a customer.

The supply network has two factories, in Liverpool and Brighton, that produce a product.  Each has a **maximum production capacity** as follows:

|Factory | Supply (tons) |
| --- | --- |
| Liverpool | 130,000 |
| Brighton |  220,000 |

The product can be shipped from a factory to a set of four depots.  Each depot has a **maximum throughput**.  Depots don't produce or consume the product; they simply pass the product on to customers.

| Depot | Throughput (tons) |
| --- | --- |
| Newcastle | 70,000 |
| Birmingham | 60,000 |
| London | 90,000 |
| Exeter | 40,000 |

The network has six customers, each with a given demand.

| Customer | Demand (tons) |
| --- | --- |
| C1 | 50,000 |
| C2 | 15,000 |
| C3 | 40,000 |
| C4 | 50,000 |
| C5 | 60,000 |
| C6 | 20,000 |

Shipping costs are given in the following table (in dollars per ton).  Columns are source cities and rows are destination cities.  Thus, for example, it costs $1 per ton to ship the product from Liverpool to London.  A '-' in the table indicates that that combination is not possible, so for example it is not possible to ship from the factory in Brighton to the depot in Newcastle.

| To | Liverpool | Brighton | Newcastle | Birmingham | London | Exeter |
| --- | --- | --- | --- | --- | --- | --- |
| Depots |
| Newcastle  | 0.5 |   - |
| Birmingham | 0.5 | 0.3 |
| London     | 1.0 | 0.5 |
| Exeter     | 0.5 | 0.2 |
| Customers |
| C1 | 1.0 | 2.0 |   - | 1.0 |   - |   - |
| C2 |   - |   - | 1.5 | 0.5 | 1.5 |   - |
| C3 | 1.5 |   - | 0.5 | 0.5 | 2.0 | 0.2 |
| C4 | 4.0 |   - | 1.5 | 1.0 |   - | 1.5 |
| C5 |   - |   - |   - | 0.5 | 0.5 | 0.5 |
| C6 | 1.0 |   - | 1.0 |   - | 1.5 | 1.5 |

The question to be answered is how to satisfy the demands of the end customers while minimizing shipping costs.

## Model Formulation

### Sets and Indices

$f \in \text{Factories}=\{\text{Liverpool}, \text{Brighton}\}$

$d \in \text{Depots}=\{\text{Newcastle}, \text{Birmingham}, \text{London}, \text{Exeter}\}$

$c \in \text{Customers}=\{\text{C1}, \text{C2}, \text{C3}, \text{C4}, \text{C5}, \text{C6}\}$

$\text{Cities} = \text{Factories} \cup \text{Depots} \cup \text{Customers}$

### Parameters 

$\text{cost}_{s,t} \in \mathbb{R}^+$: Cost of shipping one ton from source $s$ to destination $t$.

$\text{supply}_f \in \mathbb{R}^+$: Maximum possible supply from factory $f$ (in tons).

$\text{through}_d \in \mathbb{R}^+$: Maximum possible flow through depot $d$ (in tons).

$\text{demand}_c \in \mathbb{R}^+$: Demand for goods at customer $c$ (in tons).


### Decision Variables

$\text{flow}_{s,t} \in \mathbb{N}^+$: Quantity of goods (in tons) that is shipped from source $s$ to destionation $t$.

### Objective Function 

- **Cost**: Minimize total shipping costs.

\begin{equation}
\text{Minimize} \quad Z = \sum_{(s,t) \in \text{Cities} \times \text{Cities}}{\text{cost}_{s,t}*\text{flow}_{s,t}}
\end{equation}


### Constraints 

- **Factory output**: Flow of goods from a factory must respect maximum capacity.

\begin{equation}
\sum_{t \in \text{Cities}}{\text{flow}_{f,t}} \leq \text{supply}_{f} \quad \forall f \in \text{Factories}
\end{equation}

- **Customer demand**: Flow of goods must meet customer demand.

\begin{equation}
\sum_{s \in \text{Cities}}{\text{flow}_{s,c}} = \text{demand}_{c} \quad \forall c \in \text{Customers}
\end{equation}

- **Depot flow**: Flow into a depot equals flow out of the depot.

\begin{equation}
\sum_{s \in \text{Cities}}{\text{flow}_{s,d}} = \sum_{t \in \text{Cities}}{\text{flow}_{d,t}}
\quad \forall d \in \text{Depots}
\end{equation}


- **Depot capacity**: Flow into a depot must respect depot capacity.

\begin{equation}
\sum_{s \in \text{Cities}}{\text{flow}_{s,d}} \leq \text{through}_{d}
\quad \forall d \in \text{Depots}
\end{equation}


## Python 

In [6]:
## inputing libraries 
import numpy as np
import pandas as pd 
import gurobipy as gp
from gurobipy import GRB 


In [61]:
# Creating dictionaries to store the supply, throughput and customer demand limits
supply= dict({'Liverpool': 130000,'Brighton':220000})
thpt=dict({'Newcastle':70000,'Birmingham':60000,'London':90000,'Exeter':40000})
demand=dict({'C1':50000,'C2':15000,'C3':40000,'C4':50000,'C5':60000,'C6':20000})

In [62]:
# Creating dictionary to capture shipping costs over different arcs between the nodes
arcs, cost = gp.multidict({
    ('Liverpool', 'Newcastle'): 0.5,
    ('Liverpool', 'Birmingham'): 0.5,
    ('Liverpool', 'London'): 1.0,
    ('Liverpool', 'Exeter'): 0.5,
    ('Liverpool', 'C1'): 1.0,
    ('Liverpool', 'C3'): 1.5,
    ('Liverpool', 'C4'): 4.0,
    ('Liverpool', 'C6'): 1.0,
    ('Brighton', 'Birmingham'): 0.3,
    ('Brighton', 'London'): 0.5,
    ('Brighton', 'Exeter'): 0.2,
    ('Brighton', 'C1'): 2.0,
    ('Newcastle', 'C2'): 1.5,
    ('Newcastle', 'C3'): 0.5,
    ('Newcastle', 'C5'): 1.5,
    ('Newcastle', 'C6'): 1.0,
    ('Birmingham', 'C1'): 1.0,
    ('Birmingham', 'C2'): 0.5,
    ('Birmingham', 'C3'): 0.5,
    ('Birmingham', 'C4'): 1.0,
    ('Birmingham', 'C5'): 0.5,
    ('London', 'C2'): 1.5,
    ('London', 'C3'): 2.0,
    ('London', 'C5'): 0.5,
    ('London', 'C6'): 1.5,
    ('Exeter', 'C3'): 0.2,
    ('Exeter', 'C4'): 1.5,
    ('Exeter', 'C5'): 0.5,
    ('Exeter', 'C6'): 1.5
})

## Model

A model and variables are defined.The variables capture flow of materials between source and destination along an arc. While defining the varibles itself objective coeffecients (cost) are used so that there is no need of providing an optimization objective later.

In [63]:
model= gp.Model('Network Optimization')
flow=model.addVars(arcs,obj=cost,name='flow')

In [64]:
## Addressing the constraint due to supply capacity of factories 
factories= supply.keys()
factory_flow=model.addConstrs((gp.quicksum(flow.select(factory,'*'))<=supply[factory]
                               for factory in factories),name='factory')

In [65]:
## Addressing constraint to satisfy customer demands
customers=demand.keys()
customer_flow=model.addConstrs((gp.quicksum(flow.select('*',customer))==demand[customer]
                               for customer in customers),name='customer')

In [66]:
## Constraint to address flow conservation through depots
depots=thpt.keys()
depot_flow=model.addConstrs((gp.quicksum(flow.select('*',depot))==gp.quicksum(flow.select(depot,'*'))
                               for depot in depots),name='depot')

In [67]:
## Constraint to address depot throught inflow limit 
depot_capacity=model.addConstrs((gp.quicksum(flow.select('*',depot))<=thpt[depot]
                                 for depot in depots),name='depot_capacity')

In [68]:
## Model Optimization 
model.optimize()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 16 rows, 29 columns and 65 nonzeros
Model fingerprint: 0x9a229d28
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+04, 2e+05]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 15 rows, 29 columns, 64 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6550000e+05   2.062500e+04   0.000000e+00      0s
      11    2.2800000e+05   0.000000e+00   0.000000e+00      0s

Solved in 11 iterations and 0.01 seconds
Optimal objective  2.280000000e+05


**Optimal total cost is** \$  228,000. The optimal flows are as follows:

In [81]:
network_flow=pd.DataFrame(columns=['From','To','Flow'])

In [82]:

for arc in arcs:
    if flow[arc].x > 1e-6:
        network_flow = network_flow.append({"From": arc[0], "To": arc[1], "Flow": flow[arc].x}, ignore_index=True)  

        network_flow.index=['']* len(network_flow)
network_flow

,From,To,Flow
,Liverpool,Newcastle,5000.0
,Liverpool,C1,50000.0
,Liverpool,C6,20000.0
,Brighton,Birmingham,60000.0
,Brighton,London,60000.0
,Brighton,Exeter,40000.0
,Newcastle,C3,5000.0
,Birmingham,C2,15000.0
,Birmingham,C4,45000.0
,London,C5,60000.0


# Modification to the problem statement 

Two more depot options are present:

| Depot | Throughput (tons) |
| --- | --- |
| Newcastle | 70,000 |
| Birmingham | 60,000 |
| London | 90,000 |
| Exeter | 40,000 |
| Bristol | 40,000 |
| Northampton | 20,000 |

Only 4 of the six depots can be open and there is a corresponding opening cost. London and Birmingham don't have choices they have to always open

| Depot | Cost to open |
| --- | --- |
| Newcastle | 10,000 |
| Exeter | 5,000 |
| Bristol | 12,000 |
| Northampton | 4,000 |


We also have the option of expanding the capacity at Birmingham by 20,000 tons, for a cost of \$3000.

New table for shipping costs are as follows:
    
| To | Liverpool | Brighton | Newcastle | Birmingham | London | Exeter | Briston | Northhampton
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| Depots |
| Newcastle   | 0.5 |   - |
| Birmingham  | 0.5 | 0.3 |
| London      | 1.0 | 0.5 |
| Exeter      | 0.5 | 0.2 |
| Bristol     | 0.6 | 0.4 |
| Northampton | 0.4 | 0.3 |
| Customers |
| C1 | 1.0 | 2.0 |   - | 1.0 |   - |   - | 1.2 |   - |
| C2 |   - |   - | 1.5 | 0.5 | 1.5 |   - | 0.6 | 0.4 |
| C3 | 1.5 |   - | 0.5 | 0.5 | 2.0 | 0.2 | 0.5 |   - |
| C4 | 4.0 |   - | 1.5 | 1.0 |   - | 1.5 |   - | 0.5 |
| C5 |   - |   - |   - | 0.5 | 0.5 | 0.5 | 0.3 | 0.6 |
| C6 | 1.0 |   - | 1.0 |   - | 1.5 | 1.5 | 0.8 | 0.9 |
    

## New Model

## Sets and indices

$f \in \text{Factories}=\{\text{Liverpool}, \text{Brighton}\}$

$d \in \text{Depots}=\{\text{Newcastle}, \text{Birmingham}, \text{London}, \text{Exeter}, \text{Bristol}, \text{Northampton}\}$

$c \in \text{Customers}=\{\text{C1}, \text{C2}, \text{C3}, \text{C4}, \text{C5}, \text{C6}\}$

$\text{Cities} = \text{Factories} \cup \text{Depots} \cup \text{Customers}$

## Parameters 

$\text{cost}_{s,t} \in \mathbb{R}^+$: Cost of shipping one ton from source $s$ to destination $t$.

$\text{supply}_f \in \mathbb{R}^+$: Maximum possible supply from factory $f$ (in tons).

$\text{through}_d \in \mathbb{R}^+$: Maximum possible flow through depot $d$ (in tons).

$\text{demand}_c \in \mathbb{R}^+$: Demand for goods at customer $c$ (in tons).

$\text{opencost}_d \in \mathbb{R}^+$: Cost of opening depot $d$ (in dollars).

## Decision Variables 

$\text{flow}_{s,t} \in \mathbb{N}^+$: Quantity of goods (in tons) that is shipped from source $s$ to destionation $t$.

$\text{open}_{d} \in [0,1]$: Is it benefecial to keep depot $d$ open?

$\text{expand} \in [0,1]$: Is it benefecial for Birmingham to be expanded?


## Objective Function 


- **Cost**: Minimize total shipping costs plus costs of opening depots.

\begin{equation}
\text{Minimize} \quad Z = \sum_{(s,t) \in \text{Cities} \times \text{Cities}}{\text{cost}_{s,t}*\text{flow}_{s,t}} +
                          \sum_{{d} \in \text{Depots}}{\text{opencost}_d*\text{open}_d} +
                          3000 * \text{expand}
\end{equation}

## Constraints 

- **Factory output**: Flow of goods from a factory must respect maximum capacity.

\begin{equation}
\sum_{t \in \text{Cities}}{\text{flow}_{f,t}} \leq \text{supply}_{f} \quad \forall f \in \text{Factories}
\end{equation}


- **Customer demand**: Flow of goods must meet customer demand.

\begin{equation}
\sum_{s \in \text{Cities}}{\text{flow}_{s,c}} = \text{demand}_{c} \quad \forall c \in \text{Customers}
\end{equation}

- **Depot flow**: Flow into a depot equals flow out of the depot.

\begin{equation}
\sum_{s \in \text{Cities}}{\text{flow}_{s,d}} = 
\sum_{t \in \text{Cities}}{\text{flow}_{d,t}}
\quad \forall d \in \text{Depots}
\end{equation}

- **Depot capacity (except Birmingham)**: Flow into a depot must respect depot capacity, and is only allowed if the depot is open.

\begin{equation}
\sum_{s \in \text{Cities}}{\text{flow}_{s,d}} \leq \text{through}_{d} * \text{open}_{d}
\quad \forall d \in \text{Depots} - \text{Birmingham}
\end{equation}


- **Depot capacity (Birmingham)**: Flow into Birmingham must respect depot capacity, which may have been expanded.

\begin{equation}
\sum_{s \in \text{Cities}} \text{flow}_{s,\text{Birmingham}} \leq \text{through}_{\text{Birmingham}} + 20000 * \text{expand}
\end{equation}

- **Open depots**: At most 4 open depots (no choice for Birmingham or London).

\begin{equation}
\sum_{d \in \text{Depots}}{\text{open}_{d}} \leq 4
\end{equation}

\begin{equation}
\text{open}_{\text{Birmingham}} = \text{open}_{\text{London}} = 1
\end{equation}


In [93]:
## new dictionary to store the modified throughput limits
thpt1=dict({'Newcastle':70000,'Birmingham':60000,'London':90000,'Exeter':40000,'Bristol':40000,'Northampton':20000})
opencost=dict({'Newcastle': 10000,
                 'Birmingham': 0,
                 'London': 0,
                 'Exeter': 5000,
                 'Bristol': 12000,
                 'Northampton': 4000})
arcs1,cost1=gp.multidict({
    ('Liverpool', 'Newcastle'): 0.5,
    ('Liverpool', 'Birmingham'): 0.5,
    ('Liverpool', 'London'): 1.0,
    ('Liverpool', 'Exeter'): 0.5,
    ('Liverpool', 'Bristol'): 0.6,
    ('Liverpool', 'Northampton'): 0.4,
    ('Liverpool', 'C1'): 1.0,
    ('Liverpool', 'C3'): 1.5,
    ('Liverpool', 'C4'): 4.0,
    ('Liverpool', 'C6'): 1.0,
    ('Brighton', 'Birmingham'): 0.3,
    ('Brighton', 'London'): 0.5,
    ('Brighton', 'Exeter'): 0.2,
    ('Brighton', 'Bristol'): 0.4,
    ('Brighton', 'Northampton'): 0.3,
    ('Brighton', 'C1'): 2.0,
    ('Newcastle', 'C2'): 1.5,
    ('Newcastle', 'C3'): 0.5,
    ('Newcastle', 'C5'): 1.5,
    ('Newcastle', 'C6'): 1.0,
    ('Birmingham', 'C1'): 1.0,
    ('Birmingham', 'C2'): 0.5,
    ('Birmingham', 'C3'): 0.5,
    ('Birmingham', 'C4'): 1.0,
    ('Birmingham', 'C5'): 0.5,
    ('London', 'C2'): 1.5,
    ('London', 'C3'): 2.0,
    ('London', 'C5'): 0.5,
    ('London', 'C6'): 1.5,
    ('Exeter', 'C3'): 0.2,
    ('Exeter', 'C4'): 1.5,
    ('Exeter', 'C5'): 0.5,
    ('Exeter', 'C6'): 1.5,
    ('Bristol', 'C1'): 1.2,
    ('Bristol', 'C2'): 0.6,
    ('Bristol', 'C3'): 0.5,
    ('Bristol', 'C5'): 0.3,
    ('Bristol', 'C6'): 0.8,
    ('Northampton', 'C2'): 0.4,
    ('Northampton', 'C4'): 0.5,
    ('Northampton', 'C5'): 0.6,
    ('Northampton', 'C6'): 0.9
})

## Creating a new model

In [94]:
model1=gp.Model('Network Optimization Advanced')
depots1=thpt1.keys()
flow1=model1.addVars(arcs1,obj=cost,name='flow1')
open=model1.addVars(depots1,obj=cost,vtype=GRB.BINARY,name='open')
expand=model1.addVar(obj=3000,vtype=GRB.BINARY,name='expand')
open['Birmingham'].lb=1 # Resticting open_Birmingham
open['London'].lb=1


In [95]:
# Factory Constraints

factories= supply.keys()
factory1_flow = model1.addConstrs((gp.quicksum(flow1.select(factory, '*')) <= supply[factory]
                                 for factory in factories), name="factory1")

In [97]:
# Customer Constraints

customers = demand.keys()
customer1_flow = model1.addConstrs((gp.quicksum(flow1.select('*', customer)) == demand[customer]
                                  for customer in customers), name="customer1")

In [99]:
#Depot flow conservation

depot1_flow = model1.addConstrs((gp.quicksum(flow1.select(depot, '*')) == gp.quicksum(flow1.select('*', depot))
                               for depot in depots1), name="depot1")

In [100]:
# Depot throughput

# Throughput for depots other than Birmingham 

all_but_birmingham = list(set(depots1) - set(['Birmingham']))

depot1_capacity = model1.addConstrs((gp.quicksum(flow1.select(depot, '*')) <= thpt1[depot]*open[depot]
                                   for depot in all_but_birmingham), name="depot_capacity1")

In [101]:
# Throughput capacity for Birmingham 

birmingham_capacity = model1.addConstr(gp.quicksum(flow1.select('*', 'Birmingham')) <= thpt1['Birmingham'] +
                                      20000*expand, name="birmingham_capacity1")

In [102]:
# Depot count

depot1_count = model1.addConstr(open.sum() <= 4)

In [103]:
model1.optimize()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 27 rows, 49 columns and 150 nonzeros
Model fingerprint: 0xcfc7cc76
Variable types: 42 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+04]
  Objective range  [2e-01, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 2e+05]
Presolve removed 6 rows and 2 columns
Presolve time: 0.01s
Presolved: 21 rows, 47 columns, 113 nonzeros
Variable types: 42 continuous, 5 integer (5 binary)

Root relaxation: objective 1.770000e+05, 26 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    177000.00000 177000.000  0.00%     -    0s

Explored 0 nodes (26 simplex iterations) in 0.04 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 177000 

Optimal solution found (tolerance 1.00e-04)
Best objective

In [105]:
print('List of open depots:', [d for d in depots1 if open[d].x > 0.5])
if expand.x > 0.5:
    print('Expansion of Birmingham needed')

List of open depots: ['Birmingham', 'London', 'Bristol', 'Northampton']
Expansion of Birmingham needed


In [106]:
network_flow1 = pd.DataFrame(columns=["From", "To", "Flow"])
for arc in arcs1:
    if flow1[arc].x > 1e-6:
        network_flow1 = network_flow1.append({"From": arc[0], "To": arc[1], "Flow": flow1[arc].x}, ignore_index=True)  
network_flow1.index=[''] * len(network_flow1)
network_flow1

,From,To,Flow
,Liverpool,C1,50000.0
,Liverpool,C6,20000.0
,Brighton,Birmingham,80000.0
,Brighton,London,25000.0
,Brighton,Bristol,40000.0
,Brighton,Northampton,20000.0
,Birmingham,C2,10000.0
,Birmingham,C3,40000.0
,Birmingham,C4,30000.0
,London,C5,25000.0
